# Interaction Maps and the Interaction API

SuperStar uses the same underlying knowledge-base of intermolecular interaction preferences as IsoStar. It can be used to analyse potential protein binding sites or small molecules for regions where complementary interacting groups might lie. This functionality can also be accessed _via_ the [Interaction API](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/interaction_map_analysis.html).

We will also use the [Protein API](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/protein.html) to set up the binding site analysis and the [Utilities API](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/utilities.html) to postprocess the results. 

The interaction maps will be inspected in PyMOL.

In [ ]:
%run ../Discovery_Notebook_Utils.py

In [ ]:
from ccdc.protein import Protein
from ccdc.interaction import InteractionMapAnalysis
from ccdc.utilities import Grid

### Configuration

### Initialization

In [ ]:
logger.info(script_info)

### Analysis of a protein binding site

First w read a PDB file and create a protein object...

In [ ]:
pdb_file = '2UW7.pdb'

In [ ]:
protein = Protein.from_file(pdb_file)

What ligands are present in this PDB entry?

In [ ]:
[ligand.identifier for ligand in protein.ligands]

We will be examining the binding site around the ligand `A/GVP`...

In [ ]:
ligand = protein.ligands[0]

ligand.identifier

In [ ]:
mol2html(ligand)

First, we set up the binding site analysis using a `ProteinSettings` object...

In [ ]:
settings = InteractionMapAnalysis.ProteinSettings()

Set the working directory, _i.e._ where the output directory is to be created (note that an absolute path is required)...

In [ ]:
settings.working_directory = Path('.').absolute()

By default, the whole protein will be searched for cavities.

Instead, we will use a cavity based on the residues around the chosen ligand...

In [ ]:
settings.detect_cavities = False

settings.detect_cavity_from_residues = Protein.BindingSiteFromMolecule(protein, ligand, 6.0)

We will use the CSD as the source for scatterplot data...

In [ ]:
settings.source = 'CSD'

We will use all three probes available by default...

In [ ]:
settings.probe_names

We wish to generate a contour surface and rotate terminal -X-H groups on the protein (_e.g._ in Ser, Thr, Tyr, Cys and Lys residues)...

In [ ]:
settings.calculate_contour_surface = True

settings.rotate_torsions = True # For rotatable OH, SH and NH groups on the protein

Perform the analysis...

In [ ]:
%%time

analyser = InteractionMapAnalysis(settings=settings)

results = analyser.analyse_protein(protein)

Note that the protein, interaction maps and hotspots (_i.e._ maxima for the interaction maps) are saved by default.

The maps are saved in `acnt` format. This can be easily converted to other formats, such as a grid file that may be read by PyMOL...

In [ ]:
results_dir = Path(results.directory)

In [ ]:
for probe_name in settings.probe_names:
    
    name = probe_name.replace(' ', '_')  # Fix probe name
        
    grid = Grid.from_file(str(results_dir  / f'{name}.acnt'))

    grid.write(str(results_dir  / f'{name}.grd'))

Save the ligand to the results directory for visualization...

In [ ]:
ligand.identifier = ligand.identifier.replace(':', '_')  # Fix the name, as the colon can cause problems when used in file paths

with MoleculeWriter(str(results_dir / f'{ligand.identifier}.mol2')) as writer:

    writer.write(ligand)

Visualise using PyMOL...

In [ ]:
# Local utility to visualise results using PyMOL, via the helper script `visualise.py`.
# As I'm not sure how to pass parameters to a PyMOL script run this way, I'll just do it via a file.

def view_results(results_dir):
    
    pymol = start_pymol()

    if pymol:

        with Path('results_dir.txt').open('w') as file:

             file.write(str(results_dir))

        pymol.do("run visualise.py")

        os.unlink('results_dir.txt')

In [ ]:
view_results(results_dir)

### Analysis of a Small Molecule

It is also possible to analyse a small molecule, independently of a protein.

We now use the SmallMoleculeSettings class instead of the ProteinSettings class...

In [ ]:
settings = InteractionMapAnalysis.SmallMoleculeSettings()

Again, we use all probes available by default...

In [ ]:
settings.probe_names

Perform the analysis...

In [ ]:
analyser = InteractionMapAnalysis(settings=settings)

results = analyser.analyse_small_molecule(ligand)

Convert maps to grid format for PyMOL...

In [ ]:
results_dir = Path(results.directory)

In [ ]:
for probe_name in settings.probe_names:
    
    name = probe_name.replace(' ', '_')  # Fix probe name
        
    grid = Grid.from_file(str(results_dir  / f'{name}.acnt'))

    grid.write(str(results_dir  / f'{name}.grd'))

Visualise results using PyMOL, as before.

_N.B._ This is commented out so as not to interfere with the visualization of the Protein analysis. Just uncomment it as necessary.

In [ ]:
# view_results(results_dir)